In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
batting_data = pd.read_csv('mlb_batting.csv')

In [3]:
pitching_data = pd.read_csv('mlb_pitching_stats.csv')

In [4]:
batting_data.info

<bound method DataFrame.info of        name_last_first  player_id  year  player_age   ab   pa  hit  single  \
0        Hunter, Torii     116338  2015          39  521  567  125      81   
1         Ortiz, David     120074  2015          39  528  614  144      70   
2      Rodriguez, Alex     121347  2015          39  523  620  131      75   
3      Ramirez, Aramis     133380  2015          37  475  516  117      68   
4       Beltré, Adrian     134181  2015          36  567  619  163     109   
...                ...        ...   ...         ...  ...  ...  ...     ...   
1370  Chourio, Jackson     694192  2024          20  528  573  145      91   
1371   Schanuel, Nolan     694384  2024          22  519  607  130      98   
1372   Langford, Wyatt     694671  2024          22  499  557  126      81   
1373      Young, Jacob     696285  2024          24  468  521  120      92   
1374  Merrill, Jackson     701538  2024          21  554  593  162     101   

      double  triple  ...  oppo

In [5]:
pitching_data.info

<bound method DataFrame.info of        player_name_LF  player_id  year  player_age  p_game  p_formatted_ip  \
0    Wainwright, Adam     425794  2021          39      32           206.1   
1       Greinke, Zack     425844  2021          37      30           171.0   
2          Hill, Rich     448179  2021          41      32           158.2   
3     Morton, Charlie     450203  2021          37      33           185.2   
4         Lester, Jon     452657  2021          37      28           141.1   
..                ...        ...   ...         ...     ...             ...   
406     Spence, Mitch     687765  2024          26      35           151.1   
407    Harrison, Kyle     690986  2024          22      24           124.1   
408   Pfaadt, Brandon     694297  2024          25      32           181.2   
409      Stone, Gavin     694813  2024          25      25           140.1   
410      Skenes, Paul     694973  2024          22      23           133.0   

      pa   ab  hit  single  ...

In [6]:
test_data = {'A': [0, 1, 2], 'B': [0, 1, 2]}
batting_norm = pd.DataFrame()
print(batting_norm)

Empty DataFrame
Columns: []
Index: []


In [14]:
# Z normalize list of desired columns

def normalize(data, col_list):
    new_data = {}
    for column in data.columns:
        col_data = []
        if column in col_list:
            for index_r in range(data.shape[0]):    
                z_normalize = (data.loc[index_r, column] - data[column].mean()) / data[column].std()
                col_data.append(z_normalize)
            new_data[column] = col_data
    return(new_data)

In [9]:
batting_data.loc[0,'ab']

521

In [15]:
test_df = batting_data
new_data = normalize(batting_data, ['ab'])
print(test_df)

       name_last_first  player_id  year  player_age        ab   pa  hit  \
0        Hunter, Torii     116338  2015          39  0.140171  567  125   
1         Ortiz, David     120074  2015          39  0.200554  614  144   
2      Rodriguez, Alex     121347  2015          39  0.157423  620  131   
3      Ramirez, Aramis     133380  2015          37 -0.256633  516  117   
4       Beltré, Adrian     134181  2015          36  0.536975  619  163   
...                ...        ...   ...         ...       ...  ...  ...   
1370  Chourio, Jackson     694192  2024          20  0.200554  573  145   
1371   Schanuel, Nolan     694384  2024          22  0.122918  607  130   
1372   Langford, Wyatt     694671  2024          22 -0.049605  557  126   
1373      Young, Jacob     696285  2024          24 -0.317017  521  120   
1374  Merrill, Jackson     701538  2024          21  0.424834  593  162   

      single  double  triple  ...  opposite_percent  batted_ball  \
0         81      22       0  .

In [12]:
print(test_df['ab'])

0       521
1       528
2       523
3       475
4       567
       ... 
1370    528
1371    519
1372    499
1373    468
1374    554
Name: ab, Length: 1375, dtype: int64


In [16]:
for key in new_data.keys():
    test_df[key] = new_data[key]

print(test_df['ab'])

0       0.140171
1       0.200554
2       0.157423
3      -0.256633
4       0.536975
          ...   
1370    0.200554
1371    0.122918
1372   -0.049605
1373   -0.317017
1374    0.424834
Name: ab, Length: 1375, dtype: float64
